In [8]:
import pandas as pd
import numpy as np

# plotly 
import plotly.express as px
import plotly.graph_objects as go


# dashboards
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table 
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from datetime import date
from dash.exceptions import PreventUpdate
from jupyter_dash import JupyterDash

data = pd.read_csv('schedule.csv', index_col=0)


teams=data['opponent_name'].unique()
team_abbr = data['opponent_abbr'].unique()
seasons=data['season'].unique()







app = JupyterDash(__name__,external_stylesheets = [dbc.themes.BOOTSTRAP])


layout = dict(
    autosize=True,
    automargin=True,
    margin=dict(l=30, r=30, b=20, t=40),
    hovermode="closest",
    plot_bgcolor="#F9F9F9",
    paper_bgcolor="#F9F9F9",
    legend=dict(font=dict(size=10), orientation="h"),
    title="Satellite Overview",
)



app.layout = html.Div(
    [
        html.Div(
            [
                html.H2('NBA Schdule Visualization Tool', style={'color': 'lightgrey', 'fontSize': '18px'})
        ]), 

        dbc.Card([


            ## Top left box 

            dbc.CardBody([

                html.Div([html.H4('Select Team', style = {'fontSize':'18px'})]), 

                html.Div([
                    dcc.Dropdown(
                        id='team_value',
                        options=sorted([{'label': teams[i], 'value': team_abbr[i]} for i in range(len(teams))], key = lambda x: x['label']),
                        value='ATL'
                                )
                        ], style={'width': '85%', 'display': 'inline-block'}), 



                html.Div([html.H4('Select Season', style = {'fontSize':'18px'})]), 
                

                html.Div([
                    dcc.Dropdown(
                        id='season_value',
                        options=[{'label': str(int(s)-1)+'-'+ str(int(s)), 'value': s} for s in seasons],
                        value=2022
                                )
                        ], style={'width': '85%', 'float': 'center', 'display': 'inline-block'}),


                html.Div([html.H4('Select Date Range', style = {'fontSize':'18px'})]), 


                html.Div([
                dcc.DatePickerRange(
                    id='datepicker',
                    month_format='YYYY MMM'
                        )
                    ], style={'width': '85%', 'float': 'center', 'display': 'inline-block'}),

                ]),

            ],
            style={"width": "30%", 'padding': '5px 5px 5px 5px'}),

        ### Top right box

        dbc.Card([

            dbc.CardBody([
                html.Div([html.H4('INSERT MAP HERE', style = {'fontSize':'18px'})]), 

                dcc.Graph(id='map'), 




                ])
            ],
            style={'width':'68%', 'float': 'right'}), 


        ### Bottom left box
        dbc.Card([
            dbc.CardBody([
                html.Div([html.H4('INSERT GRAPHS HERE', style = {'fontSize':'18px'})]), 


            dbc.Tabs([

            
                # Home Tab
                dbc.Tab([
                    dcc.Graph(id='home_graph'),
                    ], label = 'Home'), 

                # Away Tab
                dbc.Tab([
                    dcc.Graph(id='away_graph'),
                    ], label = 'Away'), 

                # B2B Tab
                dbc.Tab([
                    dcc.Graph(id='b2b_graph'),
                    ], label = 'B2B'), 

                # 3-in-4 Tab
                dbc.Tab([
                    dcc.Graph(id='3in4_graph'),
                    ], label = '3 In 4'), 

                # 4-in-5 Tab
                dbc.Tab([
                    dcc.Graph(id='4in5_graph'),
                    ], label = '4 In 5'), 
                
                # dot graph
                dbc.Tab([
                    dcc.Graph(id='Dot_plot') 
                    ], label = 'Average Distance'),

                # line graph
                dbc.Tab([
                    dcc.Graph(id='line_plot') 
                    ], label = 'Cumulative Distance'),
            ])

            



                ])
            ],
            style={'width':'49%', 'float': 'left'}), 

        ### Bottom right box
        dbc.Card([
            dbc.CardBody([

                dash_table.DataTable(

                    id='table', 
                    style_as_list_view=True,
                    style_cell = {
                        'font_family': 'helvetica',
                        'font_size': '18px',
                        'text_align': 'center',
                        'color': 'black'
                    },
                    sort_action='native',
                    style_header={
                        'fontWeight': 'bold',
                        'backgroundColor': '#5880c4',
                        'color': 'white'
                    }, 
                    style_data_conditional=[
                        {
                            'if': {
                                'filter_query': '{Result} eq "Win"',
                                'column_id': 'Result'
                            }, 
                            'color':'green'

                        }, 
                        {
                            'if': {
                                'filter_query': '{Result} eq "Loss"',
                                'column_id': 'Result'
                            }, 
                            'color':'red'

                        } 
                    ],
                    style_table={'height': '560px', 'overflowY': 'auto'}

                    
                    )
                ])
            ],
            style={'width':'49%', 'float': 'right'}), 
    
    



]
)






@app.callback(
    Output('datepicker', 'start_date'),
    Output('datepicker', 'end_date'),
    Input('season_value', 'value')
    )

def update_date_range(season_value): 

    df = pd.read_csv('schedule.csv', index_col=0)
    df = df[df['season']==season_value]
    df = df[df['playoffs']==0]
    min_date = df['datetime'].min()
    max_date = df['datetime'].max()
    return min_date, max_date

@app.callback(
    Output('map', 'figure'),
    Input('team_value', 'value'),
    Input('season_value', 'value'),
    Input('datepicker', 'start_date'),
    Input('datepicker', 'end_date'))

def update_graph(team_value, season_value, start_date, end_date): 
    
    df=pd.read_csv('schedule.csv')
    df=df[df['team']==team_value]

    df=df[df['season']==season_value]
    df = df[(df['datetime'] > start_date) & (df['datetime'] < end_date)]

    df1=df
    # finding the locations of the team
    df1['previous_location']= np.NaN
    df1['previous_location']= df1.team_coords


    df1['current_location']=df1[['location','team_coords','opp_coords']].apply(lambda row: row['team_coords'] if row['location']=='Home' else row['opp_coords'],axis =1)
    df1['previous_location'][1:]=df1['current_location'][:-1]

    df1.reset_index(drop=True, inplace=True)

    #print(df1['previous_location'][0])

    # get list of coordinates
    list_coord= [df1['previous_location'][0]] + list(df1['current_location'])

    # get lon and lat
    lat_list=[]
    long_list=[]
    for i in range(len(list_coord)):
        lat_list.append(list_coord[i][1:list_coord[i].index(',')])
        long_list.append(list_coord[i][(list_coord[i].index(',')+2):-1])

    df1['lat']=lat_list[1:]
    df1['lon']=long_list[1:]
    df1['lat']=pd.to_numeric(df1["lat"], downcast="float")
    df1['lon']=pd.to_numeric(df1["lon"], downcast="float")
    #df1.head()

    #plotting 

    fig = px.line_mapbox(df1, lat="lat", lon="lon", zoom=3, height=300)
    fig.update_layout(mapbox_style="open-street-map", mapbox_zoom=4, mapbox_center_lat = 41,
    margin={"r":0,"t":0,"l":0,"b":0})
    
    return fig

if __name__ == '__main__':
    app.run_server('inline')


/var/folders/rs/7qrv71f13kz6q3srgk2kxw000000gn/T/ipykernel_58623/4124369681.py:267: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

